In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity


In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
interactions_df = pd.read_csv('/kaggle/input/processed/interactions_processed_kion.csv')
users_df = pd.read_csv('/kaggle/input/processed/users_processed_kion.csv')
items_df = pd.read_csv('/kaggle/input/processed/items_processed_kion.csv')

In [4]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [5]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [6]:
!mkdir recbox_data

In [7]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [25]:
from recbole.data import Interaction

In [8]:
!pip install recbole

  Obtaining dependency information for recbole from https://files.pythonhosted.org/packages/1e/d1/81756635abf971deeaa8180dae167e6ee867f9ffe13dc128a51fb9efe710/recbole-1.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: colorlog
    Found existing installation: colorlog 6.8.0
    Uninstalling colorlog-6.8.0:
      Successfully uninstalled colorlog-6.8.0
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.6
    Uninstalling colorama-0.4.6:
      Successfully uninstalled colorama-0.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bayesian-optimization 1.4.3 requires colorama>=0.4.6, but you have colorama 0.4.4 which is incompatible.


In [10]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

2023-12-13 12:05:26,777	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-13 12:05:27,390	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [11]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [9]:
!pip install kmeans_pytorch

In [12]:
dataset = create_dataset(config)
logger.info(dataset)

recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [13]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


In [14]:
import time

In [47]:
%%time
model_list = ['MultiVAE', "RecVAE",'LightGCN'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running MultiVAE...


['/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-4509a7f8-f285-48ce-ba6a-865e528f02b9.json']

General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]

It took 2.29 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.084), ('mrr@10', 0.1695), ('ndcg@10', 0.0825), ('hit@10', 0.3503), ('precision@10', 0.0467)])}
running RecVAE...


['/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-4509a7f8-f285-48ce-ba6a-865e528f02b9.json']

General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]

It took 5.81 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0849), ('mrr@10', 0.1697), ('ndcg@10', 0.0828), ('hit@10', 0.3532), ('precision@10', 0.047)])}
running LightGCN...


['/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-4509a7f8-f285-48ce-ba6a-865e528f02b9.json']

General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]

It took 23.15 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0792), ('mrr@10', 0.1685), ('ndcg@10', 0.0795), ('hit@10', 0.3385), ('precision@10', 0.0441)])}
CPU times: user 46min 40s, sys: 39.6 s, total: 47min 19s
Wall time: 31min 15s


In [17]:
result = run_recbole(model='RecVAE', dataset = 'recbox_data',config_dict = parameter_dict )

['/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-574d36fa-c46c-4c84-9f81-8eb079849012.json']

General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]

In [24]:
model = MultiVAE(config, dataset)

Max value of user's history interaction records has reached 23.254401942926535% of the total.


In [15]:
from recbole.model.general_recommender.multivae import MultiVAE

In [16]:
vae = MultiVAE(config, train_data.dataset).to(config["device"])
trainer = Trainer(config, vae)

# training
best_valid_score, best_valid_results = trainer.fit(
    train_data,
    valid_data, 
    show_progress=True,
)

# evaluation
vaeres = trainer.evaluate(test_data)

Max value of user's history interaction records has reached 20.9471766848816% of the total.
Train     0:   0%|                                                            | 0/7 [00:00<?, ?it/s]:  14%|███████▍                                            | 1/7 [00:00<00:04,  1.27it/s]:  29%|██████████████▊                                     | 2/7 [00:01<00:03,  1.54it/s]:  43%|██████████████████████▎                             | 3/7 [00:01<00:02,  1.70it/s]:  57%|█████████████████████████████▋                      | 4/7 [00:02<00:01,  1.73it/s]:  71%|█████████████████████████████████████▏              | 5/7 [00:02<00:01,  1.77it/s]:  86%|████████████████████████████████████████████▌       | 6/7 [00:03<00:00,  1.84it/s]: 100%|████████████████████████████████████████████████████| 7/7 [00:03<00:00,  2.17it/s]: 100%|████████████████████████████████████████████████████| 7/7 [00:03<00:00,  1.86it/s]
epoch 0 training [time: 3.77s, train loss: 3249.5871]
Saving current: saved/MultiVAE-Dec-13-202

In [32]:
vae.eval()

with torch.no_grad():
    scores = vae.full_sort_predict(data_l)



In [37]:
scores.shape

torch.Size([43988076])

In [48]:
13355 * 3294

43991370

In [39]:
13354 * 3294

43988076